In [1]:
# python 3 
# load library 
# op 
import pandas as pd
import numpy as np
# ML 
from sklearn.cross_validation import train_test_split, KFold, cross_val_score
from sklearn.feature_selection import RFE, f_regression
from sklearn.linear_model import (LinearRegression, Ridge, Lasso, RandomizedLasso)
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.formula.api as smf
import statsmodels.api as sm



/Users/yennanliu/anaconda3/envs/ds_dash/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/yennanliu/anaconda3/envs/ds_dash/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [15]:

# help function 
# ---------------------------------
# OP 

def load_data():
	df = pd.read_csv('kc_house_data.csv')
	return df 

def get_linearmodel_col(df):
    cols = ['price',
            'sqft_living',
            'sqft_lot',
            'sqft_above',
            'sqft_basement']
    return df[cols]
    
    
def data_clean(df):
    df_ = df[(df['price'] < df['price'].quantile(0.97))&
     (df['price'] > df['price'].quantile(0.03))]
    print (' data count (before clean) : ', len(df))
    print (' data count (before clean) : ', len(df_))
    return df_ 


def feature_extrace(df):
	pass 

def get_train_test_set(df):
    X = df[['sqft_living',
         'sqft_lot',
         'sqft_above',
         'sqft_basement']]
    y = df['price']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    return X_train,y_train, X_test, y_test


def get_train_test_set_V2(df):
    X = df.iloc[:,3:]
    y = df['price']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    return X_train,y_train, X_test, y_test
    
# ---------------------------------


# ML 

def linear_model(X_train,y_train,X_test,y_test,model):
    model.fit(X_train,y_train)
    print ('---  train score ---')
    print (model.score(X_train,y_train ))
    print ('--- intercept_  --- ')
    print(model.intercept_)
    print ('--- coef_ --- ')
    print(model.coef_)
    y_test_predict = model.predict(X_test)
    print ('--- mean_squared_error --- ')
    print (mean_squared_error(y_test_predict, y_test))
    print ('--- r2_score --- ')
    """
    Best possible score is 1.0 
    and it can be negative (because the model can be arbitrarily worse). 
    A constant model that always predicts the expected value of y, 
    disregarding the input features, would get a R^2 score of 0.0.
    """
    print (r2_score(y_test_predict, y_test))
    return y_test_predict 





def linear_model_evaluate(y_test_predict,y_test):
    # https://www.scipy-lectures.org/packages/statistics/index.html
    from matplotlib import pyplot as plt
    import seaborn as sns
    from scipy import stats
    #%matplotlib inline
    print (' --- scatter plot --- ')
    x = range(0,int(max(y_predict)))
    y = x 
    plt.scatter(np.array(y_test),y_test_predict)
    plt.plot(x,y)
    print (' --- histagram plot --- ')
    plt.hist(y_test_predict, alpha = .5)
    plt.hist(np.array(y_test) , alpha = .5)
    plt.legend(['predict', 'true'])
    print (' --- hypothesis test --- ')


def statistics_linear_model(X_train,y_train,X_test,y_test):
    # https://machinelearningmastery.com/parametric-statistical-significance-tests-in-python/
    # https://www.ritchieng.com/machine-learning-evaluate-linear-regression-model/
    """
    *** Model evaluate 
     Mean Absolute Error (MAE)
     Mean Squared Error (MSE) 
     Root Mean Squared Error (RMSE)
    """
    results = sm.OLS(y_train, X_train).fit()
    print(results.summary())
    y_test_pred = results.predict(X_test)
    print (np.sqrt(mean_squared_error(y_test, y_test_pred)))
    print (r2_score(y_test_pred, y_test))




In [13]:
# model V1 
df = load_data()
df_ = data_clean(df)
df_ = get_linearmodel_col(df_)
X_train,y_train, X_test, y_test = get_train_test_set(df_)
statistics_linear_model(X_train,y_train, X_test, y_test)

 data count (before clean) :  21613
 data count (before clean) :  20311
                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.888
Model:                            OLS   Adj. R-squared:                  0.888
Method:                 Least Squares   F-statistic:                 3.596e+04
Date:                Mon, 16 Jul 2018   Prob (F-statistic):               0.00
Time:                        03:31:09   Log-Likelihood:            -1.8444e+05
No. Observations:               13608   AIC:                         3.689e+05
Df Residuals:                   13605   BIC:                         3.689e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------

## Comments : model V1 
This is a simple linear model only train with *** 4 variables (sqft_living/ sqft_lot / sqft_above / sqft_basement)***  which are the variables I think may be strong co-releated to the prices. data preprocess includes one data cleaning process, no feature engineering / re-scale / super parameter tuning  ..
> - sqft_living/ sqft_above / sqft_basement are  positively associated with prices 
- sqft_lot is slightly negatively associated with prices 
- sqft_living/ sqft_lot / sqft_above / sqft_basement all have small P-values
-  The linear model works OK on train set (R-squared =  0.888), but not work very well on test set (R-squared =  0.022). Need to re-check data quality such as outlier/feature selection / multicollinearity ... 

In [14]:
# model V2 
df = load_data()
df_ = data_clean(df)
X_train,y_train, X_test, y_test = get_train_test_set_V2(df_)
statistics_linear_model(X_train,y_train, X_test, y_test)


 data count (before clean) :  21613
 data count (before clean) :  20311
                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.947
Model:                            OLS   Adj. R-squared:                  0.947
Method:                 Least Squares   F-statistic:                 1.429e+04
Date:                Mon, 16 Jul 2018   Prob (F-statistic):               0.00
Time:                        03:31:15   Log-Likelihood:            -1.7934e+05
No. Observations:               13608   AIC:                         3.587e+05
Df Residuals:                   13591   BIC:                         3.589e+05
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------

## Comments :  model V2
This is a  linear model  train with *** all variables which is the comparison model with the V1 above ***. data preprocess includes one data cleaning process, no feature engineering / re-scale / super parameter tuning  ..
> - sqft_living/ sqft_above / sqft_basement / sqft_living15 / bathrooms / condition.. are positively associated with prices 
- Variables like bedrooms/ yr_built / ... are srong negatively associated with prices which is intertesting part to dig in  
- All variables have small P-values
-  The linear model works "too good"  on train set (R-squared =  0.947), need to double check if "over-fitting" on 
train set, one of the clue is the "multicollinearity" warnings from model output
- The linear model works better in test set than V1 model above(R-squared =  0.53), but still not good enough.
Will need to do much data clean/ feature engineeting to optimize the linear model 
Or,  can try some non-linear model instead (i.e. SVM/DT/ ...) (in case non-linear data )